# 🎯 Tutorial Práctico: Saiku Analytics con DM_ACADEMICO

## 🎯 Objetivos
- Configurar Saiku con Oracle usando scripts automatizados
- Crear cubos OLAP multidimensionales
- Realizar análisis interactivos con datos académicos reales
- Crear visualizaciones y dashboards ejecutivos

## 🚀 ¿Qué es Saiku?
**Saiku** es una herramienta de BI que permite análisis visual drag & drop sobre cubos OLAP. Perfecta para usuarios que quieren explorar datos sin escribir código.

---

## 🔧 PASO 1: Configuración Automática de la Base de Datos

### 1.1 Ejecutar Script de Configuración CDB
```powershell
# Desde el directorio raíz del proyecto
cd saiku-setup
.\setup_saiku_cdb.ps1
```

**Este script:**
- ✅ Crea usuario `C##SAIKU_USER` en Oracle CDB
- ✅ Configura database links hacia XEPDB1
- ✅ Crea sinónimos para acceder a las tablas originales
- ✅ Verifica conectividad

### 1.2 Crear Vistas Optimizadas para OLAP
```powershell
# Ejecutar después del script anterior
.\create_saiku_views.ps1
```

**Este script crea:**
- 📊 `V_HECHOS_MATRICULA`: Tabla de hechos con campos calculados
- 📐 `V_DIM_CENTRO`: Dimensión institucional con jerarquías
- 📚 `V_DIM_PLAN_ESTUDIO`: Dimensión académica optimizada
- 📅 `V_DIM_CURSO_ACADEMICO`: Dimensión temporal
- 👥 `V_DIM_SEXO`: Dimensión demográfica
- 📈 `V_RESUMEN_EJECUTIVO`: Vista agregada para dashboards

### ✅ Verificación
Si ambos scripts se ejecutan exitosamente, tendrás todo configurado para Saiku.

## 🚀 PASO 2: Configurar Pentaho para Saiku

### 2.1 Iniciar Pentaho Server
```bash
cd pentaho-server
start-pentaho.bat  # Windows
./start-pentaho.sh # Linux/Mac
```

### 2.2 Acceder a Pentaho
1. Abrir navegador: `http://localhost:8080/pentaho`
2. **Credenciales:**
   - Usuario: `admin`
   - Contraseña: `password`

### 2.3 Configurar Data Source
1. Ir a **"Manage Data Sources"**
2. **"New Connection"** con estos parámetros:
```properties
Connection Name: DM_ACADEMICO_SAIKU
Connection Type: Oracle
Access: Native (JDBC)
Host Name: localhost
Port: 1521
Database Name: XE
User Name: C##SAIKU_USER
Password: saiku_password
```
3. **"Test"** para verificar conexión
4. **"OK"** para guardar

## 🧊 PASO 3: Crear Cubo OLAP con Data Model

### 3.1 Crear Nuevo Modelo
1. En Pentaho, ir a **"File" → "New" → "Data Source"**
2. Seleccionar connection **"DM_ACADEMICO_SAIKU"**
3. Seleccionar Source Type: Database Table(s)
4. En "Create data source for:", seleccionar: "Reporting and Analysis"
5. Seleccionar "Next"

### 3.2 Configurar Joins Manualmente


#### Tabla de Hechos (Fact Table):
- **`V_HECHOS_MATRICULA`** (centro del star schema)

#### Joins con Dimensiones:
```sql
-- Join Centro
V_HECHOS_MATRICULA.ID_CENTRO = V_DIM_CENTRO.ID_CENTRO

-- Join Plan de Estudio  
V_HECHOS_MATRICULA.ID_PLAN_ESTUDIO = V_DIM_PLAN_ESTUDIO.ID_PLAN_ESTUDIO

-- Join Curso Académico
V_HECHOS_MATRICULA.ID_CURSO_ACADEMICO = V_DIM_CURSO_ACADEMICO.ID_CURSO_ACADEMICO

-- Join Sexo
V_HECHOS_MATRICULA.ID_SEXO = V_DIM_SEXO.ID_SEXO
```

### 3.3 Configurar Medidas
**Medidas principales:**
- `CONTADOR_MATRICULAS` (COUNT)
- `CREDITOS` (SUM)
- `EDAD` (AVG)
- `ID_ALUMNO` (DISTINCT COUNT)

### 3.4 Guardar Modelo
1. **"File" → "Save As"**
2. Nombre: **"DM_Academico_OLAP"**
3. **"Publish"** para hacer disponible en Saiku

## 📊 PASO 4: Crear Análisis en Saiku Analytics

### 4.1 Abrir Saiku Analytics
1. En Pentaho, ir a **"File" → "New" → "Saiku Analytics"**
2. Seleccionar el cubo **"DM_Academico_OLAP"**
3. Se abrirá la interfaz drag & drop

### 4.2 Interfaz de Saiku
```
┌─────────────────┬───────────────────┐
│  📁 Dimensions  │  📈 Query Result  │
│  ├─ Centro      │                   │
│  ├─ Plan Est.   │                   │
│  ├─ Curso Acad. │                   │
│  └─ Sexo        │                   │
│                 │                   │
│  📈 Measures    │                   │
│  ├─ Matrículas  │                   │
│  ├─ Créditos    │                   │
│  └─ Edad Prom.  │                   │
└─────────────────┴───────────────────┘
```

## 🎯 PASO 5: Ejercicios Prácticos de Análisis

### 📊 Ejercicio 1: Análisis de Matrículas por Año
**Configuración:**
- **ROWS:** Arrastrar `Curso_Academico` → `Nombre_Curso_Academico`
- **MEASURES:** Arrastrar `Contador_Matriculas`
- **Ejecutar:** Botón "Run Query"

**Resultado esperado:**
```
Año Académico     │ Total Matrículas
──────────────────┼─────────────────
2020/2021         │ 15,234
2021/2022         │ 16,789
2022/2023         │ 17,456
```

### 📈 Ejercicio 2: Análisis Cruzado Género por Centro
**Configuración:**
- **ROWS:** `Centro` → `Nombre_Tipo_Centro`
- **COLUMNS:** `Sexo` → `Genero_Normalizado`
- **MEASURES:** `Contador_Matriculas`

**Análisis:** Distribución de género por tipo de centro académico

### 🎨 Ejercicio 3: Visualización con Gráficos
1. **Crear consulta** de matrículas por tipo de estudio
2. **Cambiar vista:** Clic en icono 📊 "Bar Chart"
3. **Personalizar:** Colores, títulos, etiquetas
4. **Exportar:** PNG, PDF, o Excel

## 📈 PASO 6: Dashboard Ejecutivo

### 6.1 Crear Dashboard Multi-Panel
**Panel 1: KPIs Principales**
- Total estudiantes únicos
- Promedio de edad
- Promedio de créditos

**Panel 2: Distribución por Género**
- Gráfico circular M/F
- Filtro por año académico

**Panel 3: Evolución Temporal**
- Gráfico de líneas por año
- Series por tipo de centro

**Panel 4: Ranking Centros**
- Top 10 centros por estudiantes
- Drill-down a campus

### 6.2 Funcionalidades Avanzadas
```mdx
-- Ejemplo: Top 5 centros por matrículas
TopCount([Centro].[Nombre_Centro].Members, 5, [Measures].[Contador_Matriculas])

-- Filtro: Solo centros con más de 1000 estudiantes
Filter([Centro].[Nombre_Centro].Members, [Measures].[Contador_Matriculas] > 1000)
```

### 6.3 Exportación y Compartición
- **PDF:** Informes ejecutivos
- **Excel:** Análisis detallado
- **PNG:** Gráficos para presentaciones
- **Embed:** Integración en portales web

## 🚀 PASO 7: Tips y Mejores Prácticas

### 🔧 Optimización de Rendimiento
```sql
-- Las vistas ya incluyen optimizaciones:
- Campos calculados (grupos de edad, tipo dedicación)
- Jerarquías predefinidas (centro → campus → tipo)
- Agregaciones pre-calculadas (V_RESUMEN_EJECUTIVO)
```

### 🎯 Navegación Eficiente
- **Drill-down:** Doble clic para expandir jerarquías
- **Filtros:** Usar botón derecho para filtrar valores
- **Ordenación:** Arrastrar medidas para ordenar automáticamente
- **Pivoting:** Mover dimensiones entre ROWS/COLUMNS

### ⚠️ Solución de Problemas Comunes
```
❌ "No data found":
   💡 Verificar joins en Data Model
   💡 Comprobar permisos de C##SAIKU_USER

❌ "Connection timeout":
   💡 Verificar que Oracle está ejecutándose
   💡 Usar vistas agregadas para consultas grandes

❌ "Schema not found":
   💡 Re-publicar el Data Model
   💡 Reiniciar Pentaho Server
```

## ✅ Resumen: Lo que has Conseguido

### 🏆 Competencias Adquiridas
- ✅ **Configuración automática** de Saiku con Oracle
- ✅ **Creación de cubos OLAP** con Data Model
- ✅ **Análisis drag & drop** sin código SQL
- ✅ **Visualizaciones interactivas** profesionales
- ✅ **Dashboards ejecutivos** para toma de decisiones

### 🎯 Casos de Uso Dominados
```
📚 Gestión Académica:
   ✓ Análisis de matrículas por centro/año
   ✓ Distribución demográfica estudiantes
   ✓ Seguimiento tendencias crecimiento

🏛️ Administración:
   ✓ Dashboards ejecutivos rectorado
   ✓ KPIs institucionales tiempo real
   ✓ Análisis comparativo centros
```

### 🚀 Próximos Pasos
1. **Ampliar análisis** con tablas de rendimiento académico
2. **Crear alertas automáticas** para KPIs críticos
3. **Integrar con sistemas** gestión académica
4. **Desarrollar reportes personalizados** por departamento

---

🎉 **¡Felicitaciones!** Ahora dominas Saiku Analytics para análisis académicos profesionales.